In [1]:
import re #regex
import os #operating system
import pandas as pd
import numpy as np

In [2]:
directory = 'csv/perf_only_algorithm'
files = []
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if (re.match("^fwa_dev_v_.*", filename)) and (not re.match("^fwa_dev_v_.*_pitch.*", filename)):
        files.append(file_path)
        print(file_path)

csv/perf_only_algorithm/fwa_dev_v_1_1__n_030__b_02__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_030__b_03__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_030__b_05__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_030__b_06__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_030__b_10__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_050__b_10__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_057__b_03__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_057__b_19__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_110__b_02__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_110__b_05__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_110__b_10__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_110__b_11__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_1__n_110__b_22__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_2__n_030__b_02__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_2__n_030__b_03__t_01.csv
csv/perf_only_algorithm/fwa_dev_v_1_2__n_030__b_05__t_01.csv
csv/perf_only_algorithm/

In [3]:
# for filename in files :
#     f = open(filename, "r")
#     print(f.read())
#     f.close()

In [4]:
def obtain_parameter(in_string, par_name) :
    par_regex = "-" + par_name + "=[^ \n]*"
    par_string = re.search(par_regex, in_string).group()
    par_value = re.sub(".*=", "", par_string)
    return par_value

def obtain_version(in_string) :
    par_regex = "v_[^. ]*"
    par_string = re.search(par_regex, in_string).group()
    par_value = re.sub("v_", "", par_string)
    return par_value

In [5]:
def obtain_all_parameters(in_file) :
    with open(in_file) as fp :
        for i, line in enumerate(fp) :
            if i == 0 :
                n = int(obtain_parameter(line, "n"))
                b = int(obtain_parameter(line, "b"))
                t = int(obtain_parameter(line, "t"))
                version = obtain_version(line)

            elif i == 3 :
                headers = line.replace('"', '').replace("\n", "").split(",")

            elif i == 4 :
                temp = line.replace("\n", "").replace("%", "").split(",")
                unit_measures = []
                for unit in temp :
                    if unit != "" :
                        unit = "(" + unit + ")"
                    unit_measures.append(unit)

            elif i > 4 :
                break

    headers = list(map(lambda x, y: x + y, headers, unit_measures))

    return n, b, t, version, headers

In [6]:
df_big = pd.DataFrame()     # collects all data
df_small = pd.DataFrame()   # sums up everything
for filename in files : 
    n, b, t, version, headers = obtain_all_parameters(filename)
    temp_df_big = pd.read_csv(filename, skiprows=5, names=headers)
    temp_df_big['version'] = version
    temp_df_big['n'] = n
    temp_df_big['b'] = b
    temp_df_big['t'] = t
    if df_small.empty :
        df_big = pd.DataFrame(columns = headers)
        df_small = pd.DataFrame(columns = ['version', 'n', 'b', 't', 'Time(us)'])
    df_big = pd.concat([df_big, temp_df_big])
    tot_time = temp_df_big['Time(us)'].sum()
    #print(f"version = {version}, n = {n}, b = {b}, t = {t}, tot_time = {tot_time:.2f}")
    df_small.loc[-1] = [version, n, b, t, tot_time]  # adding a row
    df_small.index = df_small.index + 1  # shifting index
    df_small = df_small.sort_index()  # sorting by index
df_small = df_small.sort_values(by = ['n', 'b', 't', 'Time(us)'])


In [7]:
#print(df_small.to_string())

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_small)

   version    n   b  t     Time(us)
64     1_2   30   2  1     2641.303
51     1_3   30   2  1     2783.261
12     2_2   30   2  1     2785.047
25     2_0   30   2  1     2799.683
38     1_4   30   2  1     4129.304
77     1_1   30   2  1    25508.528
63     1_2   30   3  1     2303.718
24     2_0   30   3  1     2626.708
37     1_4   30   3  1     2696.150
11     2_2   30   3  1     2795.235
50     1_3   30   3  1     2835.764
76     1_1   30   3  1     9396.469
62     1_2   30   5  1     2359.137
36     1_4   30   5  1     2490.960
49     1_3   30   5  1     2519.385
10     2_2   30   5  1     2545.432
23     2_0   30   5  1     2818.968
75     1_1   30   5  1     4165.350
48     1_3   30   6  1     2140.728
61     1_2   30   6  1     2318.036
22     2_0   30   6  1     2502.692
35     1_4   30   6  1     2695.475
74     1_1   30   6  1     3191.344
9      2_2   30   6  1     7026.460
21     2_0   30  10  1     2113.064
34     1_4   30  10  1     2337.152
73     1_1   30  10  1     2

In [8]:
na_columns = df_big.columns[df_big.isna().any()].tolist()
if na_columns == [] :
    print("No NaN values found")
else :
    print("Columns with NaN:")
    print(na_columns)

No NaN values found
